In [163]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import nn
from matplotlib.pyplot import imread
from skimage.transform import resize
from skimage import img_as_ubyte
from PIL import Image
import torchvision.transforms as transforms



In [204]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [205]:
print(model)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [206]:
IMG_PATH = "./kite.jpg"

In [207]:
device = torch.device("cpu")

In [208]:
def output(my_model, image_path):
    img = imread(image_path)
    img = img_as_ubyte(resize(img, (256,256)))
    temp = img
    img = img.transpose(2, 0, 1)
    img = img / 255
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)
    # temp = image.transpose(1,2,3)
    # plt.imshow(temp)

    # Encode
    image = image.unsqueeze(0)  # (1, 3, 256, 256)
    res = my_model(image)  # (1, enc_image_size_h, enc_image_size_w, encoder_dim)
    return res

In [209]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [210]:
print(output(model, IMG_PATH))

[{'boxes': tensor([[  9.6999,   0.0000, 249.5541, 206.5705],
        [  3.0608, 192.9843, 185.3708, 249.1189],
        [ 34.7594,  71.0386, 256.0000, 244.6354],
        [  0.0000, 152.3233, 222.7533, 241.6259],
        [ 20.4340,   0.7897, 206.4147, 187.3404],
        [  0.0000,  52.8313, 197.6743, 235.2217],
        [ 10.2378,  74.0044, 256.0000, 196.0607]], grad_fn=<StackBackward>), 'labels': tensor([ 3,  3,  3,  3, 70,  1, 81]), 'scores': tensor([0.1631, 0.1078, 0.0772, 0.0579, 0.0550, 0.0524, 0.0502],
       grad_fn=<IndexBackward>)}]


In [211]:
"""
layers_1 = list(model.children())[:-1]
layers_2 = list(model.children())[-1].box_roi_pool
model1= nn.Sequential(
    *layers_1)
""" 

'\nlayers_1 = list(model.children())[:-1]\nlayers_2 = list(model.children())[-1].box_roi_pool\nmodel1= nn.Sequential(\n    *layers_1)\n'

In [212]:
in_features = model.roi_heads.box_predictor.cls_score.in_features

In [213]:
print(in_features)

1024


In [242]:
class Tail(nn.Module):

    def __init__(self, in_channels, encoder_embedding_dim=196):
        super(Tail, self).__init__()
        self.linear1 = nn.Linear(in_channels, 8192)
        self.linear2 = nn.Linear(8192, encoder_embedding_dim)
        self.conv_layer= nn.Conv2d(1000,2048, 1)

    def forward(self, x):
        #if x.dim() == 4:
           #assert list(x.shape[2:]) == [1, 1]
        x = x.flatten(start_dim=1)
        x = self.linear1(x)
        x = self.linear2(x)
        
        #Here, res is 1000x4096
        
        x=torch.reshape(x,(1000,14,14))
        print(x.shape)
        
        x=torch.unsqueeze(x,0)
        print(x.shape)
        #Of dimensions 2048x14x14
        res= self.conv_layer(x)

        
        

        return res,res


In [243]:
model.roi_heads.box_predictor = Tail(in_features, 196)


In [244]:
print(model)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequent

In [245]:
print(output(model,IMG_PATH ))

torch.Size([1000, 14, 14])
torch.Size([1, 1000, 14, 14])


RuntimeError: shape '[1000, -1]' is invalid for input of size 401408

In [196]:
class my_Faster_RCNN:
    def __init__():
        self.root= nn.S
        
    def forward(x):
        
        
    def backward():
        pass

IndentationError: expected an indented block (<ipython-input-196-1b0c6a707555>, line 8)